## Extract 2 tables from the article: List_of_FIFA_World_Cup_finals

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests

url = 'https://en.wikipedia.org/wiki/List_of_FIFA_World_Cup_finals'
response = requests.get(url)
print(response)

# create a BeautifulSoup object and fill the content from our url
soup = BeautifulSoup(response.content, 'html.parser')
soup.title.string

<Response [200]>


'List of FIFA World Cup finals - Wikipedia'

In [2]:
# get the html structure of the page
# print(soup.prettify())

final_matches = soup.find("table", class_='sortable')
final_matches

# collect raw rows from table
rows = final_matches.findAll("tr")

# header attributes of the table
headers = [th.text.rstrip() for th in rows[0].find_all('th')]

# extract a list of years
years = []

for year in rows:
    data = [th.text.rstrip() for th in year.find_all('th')]
    years.append(data)
    
year = [j for i in years for j in i]
matchyear = year[8:-11]

# extract all rows from table
allRows = []

for row in rows:
    data = [td.text.rstrip() for td in row.find_all('td')]
    
    allRows.append(data)
allRows = allRows[1:-4]

# create a dataframe from collected data
allRows = pd.DataFrame(allRows, columns=headers[1:])
finales = allRows.copy()
finales['Years'] = matchyear
finales = finales[['Years', 'Winners', 'Score[2]', 'Runners-up', 'Venue', 'Location', 'Attendance']]
finales.columns = ['Year', 'Winners', 'Score', 'Runners-up', 'Venue', 'Location', 'Attendance',]
finales

,Year,Winners,Score,Runners-up,Venue,Location,Attendance
0,1930,Uruguay,4–2,Argentina,Estadio Centenario,"Montevideo, Uruguay","68,346"
1,1934,Italy,2–1 (a.e.t.),Czechoslovakia,Stadio Nazionale PNF,"Rome, Italy","55,000"
2,1938,Italy,4–2,Hungary,Stade Olympique de Colombes,"Colombes (Paris), France","45,000"
3,1950,Uruguay,2–1[n 3],Brazil,Maracanã Stadium,"Rio de Janeiro, Brazil","173,850"
4,1954,West Germany,3–2,Hungary,Wankdorf Stadium,"Bern, Switzerland","62,500"
5,1958,Brazil,5–2,Sweden,Råsunda Stadium,"Solna (Stockholm), Sweden","51,800"
6,1962,Brazil,3–1,Czechoslovakia,Estadio Nacional,"Santiago, Chile","69,000"
7,1966,England,4–2 (a.e.t.),West Germany,Wembley Stadium,"London, England","96,924"
8,1970,Brazil,4–1,Italy,Estadio Azteca,"Mexico City, Mexico","107,412"
9,1974,West Germany,2–1,Netherlands,Olympiastadion,"Munich, West Germany","75,200"


In [3]:
total_results = soup.find("table", class_='wikitable plainrowheaders sortable')
total_results

# collect raw rows from table
rows = total_results.findAll("tr")

# header attributes of the table
headers = [th.text.rstrip() for th in rows[0].find_all('th')]

# extract a list of teams
teams = []

for team in rows:
    data = [th.text.rstrip() for th in team.find_all('th')]
    teams.append(data)
    
team = [j for i in teams for j in i]
team = team[6:]
team

# remove reduntant characters from data
characters = str.maketrans("", "", "\xa0")
finalteams = []
finalteams = [s.translate(characters) for s in team]

# extract all rows from table
allRows2 = []

for row in rows:
    data = [td.text.rstrip() for td in row.find_all('td')]
    
    allRows2.append(data)
allRows2 = allRows2[1:]

# create a dataframe from collected data
allRows2 = pd.DataFrame(allRows2, columns=headers[1:])
results = allRows2.copy()
results['Team'] = finalteams

results = results[['Team', 'Winners', 'Runners-up', 'Total finals', 'Years won', 'Years runners-up']]
results

,Team,Winners,Runners-up,Total finals,Years won,Years runners-up
0,Brazil,5,2,7,"1958, 1962, 1970, 1994, 2002","1950, 1998"
1,Germany,4,4,8,"1954, 1974, 1990, 2014","1966, 1982, 1986, 2002"
2,Italy,4,2,6,"1934, 1938, 1982, 2006","1970, 1994"
3,Argentina,2,3,5,"1978, 1986","1930, 1990, 2014"
4,France,2,1,3,"1998, 2018",2006
5,Uruguay,2,0,2,"1930, 1950",–
6,Spain,1,0,1,2010,–
7,England,1,0,1,1966,–
8,Netherlands,0,3,3,–,"1974, 1978, 2010"
9,Czechoslovakia,0,2,2,–,"1934, 1962"


So we got 2 tables: 

* **finales**: details about 20 finales from 1930-2018 <br>
* **results**: total result of FIFA World Cup champions